<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 4. Interlinked Plots</h2></div>

Using hvPlot allows you to generate a number of different types of plot quickly from a standard API, returning [HoloViews](https://holoviews.org) objects as discussed in the previous notebook. These objects are rendered with Bokeh, which offers a number of standard ways to interact with your plot, such as panning and zooming
tools.

Many other modes of interactivity are possible when building an exploratory visualization (such as a dashboard) and these forms of interactivity cannot currently be achieved using hvPlot alone.

In this notebook, we will drop down to the HoloViews level of representation to build a visualization directly that consists of linked plots that update when you interactivity select a particular set of earthquakes with the
mouse. The goal is to show how more sophisticated forms of interactivity can be built when needed, in a way that's fully compatible with all the examples shown in earlier sections.

First let us load our initial imports:

In [ ]:
import holoviews as hv
import pandas as pd
import dask.dataframe as dd
import hvplot.pandas  # noqa
from holoviews.util.transform import lon_lat_to_easting_northing
from holoviews.element import tiles
import colorcet as cc

And clean the data before filtering (for magnitude `>7`) and projecting to to Web Mercator as before:

In [ ]:
df = dd.read_parquet('../data/earthquakes.parq').repartition(npartitions=4)
cleaned_df = df.copy()
cleaned_df['mag'] = df.mag.where(df.mag > 0)

In [ ]:
%%time
cleaned_reindexed_df = cleaned_df.set_index(cleaned_df.time)

In [ ]:
%%time
cleaned_reindexed_df = cleaned_reindexed_df.persist()

In [ ]:
most_severe = cleaned_reindexed_df[cleaned_reindexed_df.mag >= 7].compute()
x, y = lon_lat_to_easting_northing(most_severe.longitude, most_severe.latitude)
most_severe_projected = most_severe.join([pd.DataFrame({'easting': x}), pd.DataFrame({'northing': y})])

Towards the end of the previous notebook we generated a scatter plot of earthquakes across the earth with magnitudes `>7` that was projected using Datashader and overlaid on top of a map tile source:

In [ ]:
high_mag_quakes = most_severe_projected.hvplot.points(x='easting', y='northing', c='mag', 
                                                      title='Earthquakes with magnitude >= 7')
esri = tiles.ESRI().redim(x='easting', y='northing')
esri * high_mag_quakes

We also saw how this plot is actually a HoloViews `Points` object:

In [ ]:
print(high_mag_quakes)

This object is an example of a HoloViews `Element`, which is an object that can display itself. Elements are *thin* wrappers around your data, and the raw input data is always available on the `.data` attribute. For instance, we can look at the `head` of the `most_severe_projected` `DataFrame` as follows:

In [ ]:
high_mag_quakes.data.head(2)

If we wanted to learn all of HoloViews syntax, instead of using hvPlot we could actually build the equivalent visualization by creating HoloViews elements explicitly, overlaying then with the `*` operator, and setting HoloViews options with `.opts()`: 

In [ ]:
esri * hv.Points(most_severe_projected, ['easting', 'northing'], 'mag').opts(color='mag', size=8, aspect='equal')

### A brief introduction to `.opts()`

In the example at the end of the last notebook, you may have noticed the use of `.opts(bgcolor='black')` to give our population raster a dark background. In the cell above, you can see this method is used again with `.opts(color='mag', size=8, aspect='equal')`. How are these settings different from those supplied in the calls to `.hvplot`?

Basically, hvPlot translates option values you supply to `.hvplot()` calls into HoloViews options settings that it applies to the HoloViews objects it creates. Most of the time, you won't need to worry about HoloViews options directly, but occasionally there are settings that are only available at the HoloViews level (such as `bgcolor='black'` in the earlier example). In these cases, you may be able to achieve the required flexibility by using the `.opts` method on the output from hvPlot, making hvPlot a shortcut rather than a dead end even when you need more customization.

## The HoloViews `help` system

You can learn more about the `.opts` method and the HoloViews options
system in the [corresponding user
guide](http://holoviews.org/user_guide/Applying_Customizations.html). To
easily learn about the available options from inside a notebook, you can
use `hv.help` and inspect the 'Style Options'.

In [ ]:
# Commented as there is a lot of help output!
# hv.help(hv.Points) 

#### Exercise

Try using `hv.help` to inspect the options available for different element types such as an `Image` element. Copy the code cell above into the cell below and pick a `Points` option that makes sense to you and try using it in the `.opts` method.

<details><summary><i><u>(Hint)</u></i></summary><br>

If you can't decide on an option to pick, a good choice is `marker`. For instance, try:

 * `marker='+'` 
 * `marker='d'`.
 
 HoloViews uses [matplotlib's conventions](https://matplotlib.org/3.1.0/api/markers_api.html) for specifying the various marker types. Try finding out which ones are supported by Bokeh.

</details>

## Linked brushing across elements

In the previous notebook, we saw how plot axes are linked for panning and zooming when using the `+` operator provided the dimensions match. When dimensions match across plots of a single dataset, we can use a similar principle to achieve [linked brushing](http://holoviews.org/user_guide/Linked_Brushing.html).

To illustrate, let us generate two histograms from our `most_severe_projected` DataFrame:


In [ ]:
mag_hist = most_severe_projected.hvplot(
    y='mag', kind='hist', responsive=True, min_height=200)

depth_hist = most_severe_projected.hvplot(
    y='depth', kind='hist', responsive=True, min_height=200)

These two histograms are plotting two different dimensions of our earthquake dataset (magnitude and depth), derived from the same set of earthquake samples. The samples between these two histograms implicitly share an index, which allows us to enable linked brushing as follows:

In [ ]:
ls = hv.link_selections.instance()
ls(depth_hist)+ ls(mag_hist)

Try using the first Bokeh tool to select areas of either histogram: you'll then see both the depth and magnitude distributions for the bins you have selected, compared to the overall distribution. By default, selections on both histograms are combined so that the selection is the intersection of the two regions selected (data points matching _both_ the constraints on depth and the constraints on magnitude that you select). For instance, try selecting the deepest earthquakes (around 600), and you can see that those are not specific to one particular magnitude. You can then further select a particular magnitude range, and see how that range is distributed in depth over the selected depth range. Linked selections like this make it feasible to look at specific regions of a multidimensional space and see how the properties of those regions compare to the properties of other regions.

Here `ls` declares an object representing our linked selection. Passing both histograms to `ls` declares their linkage, and we can then put them in a layout with the `+` operator. (Equivalently, we could have done `ls(depth_hist+ mag_hist)`, which traverses the layout and links whatever it finds there.)

Note that these two histograms are derived from the same `DataFrame`, but this isn't necessary to achieve the linked behavior! If linking two different `DataFrames`, the important thing to check is that the columns/index are consistent and that the plots you are visualizing make sense when linked together.

## Linked brushing across element types

The previous example linked across two histograms, but nothing prevents you from linked brushing across different element types. Here are our earthquake points, also derived from the same `DataFrame` where the only change from earlier is that we are using the warm colormap (described in the previous notebook):

In [ ]:
geo = most_severe_projected.hvplot(
    'easting', 'northing', color='mag', kind='points',
    xaxis=None, yaxis=None, responsive=True, min_height=500, cmap = cc.CET_L4[::-1])

Once again, we just need to pass our points to the `ls` object (newly declared here to be independent of the one above) to declare the linkage:

In [ ]:
ls = hv.link_selections.instance()
((hv.element.tiles.ESRI() * ls(geo)) + ls(depth_hist))

Now you can select earthquakes on the map and view their corresponding depth distribution, or vice versa.

## Accessing the data selection

If you pass your `DataFrame` into the `.filter` method of your linked selection object, you can apply the active filter that you specified interactively:

In [ ]:
ls.filter(most_severe_projected)

Try selecting a small number of earthquakes on the map above and re-running the previous cell. You should see that your `DataFrame` only includes the earthquakes you have selected. You can use this linked selections feature in your own workflows by selecting a region of your data, then running subsequent analyses only on that subset of the data (or comparing that subset to the whole data set).

## Conclusion

When exploring data it can be convenient to use the `.plot` API to quickly visualize a particular dataset. By calling `.plot` to generate different plots over the course of a session, it is possible to gradually build up a mental model of how a particular dataset is structured. While this works well for simple datasets, it can be more efficient to build a linked visualization with support for direct user interaction as a tool for more rapidly gaining insight.

In particular, with interlinked plots you can:

1. Interactively explore high-dimensional data by making selections across different views of the same underlying samples.
2. Turn this interactive exploration into a subselection of your data, allowing you to continue your data analysis on a subset of your data that you interactively selected.

In the next section we will see how to apply data processing in a pipelined form, allowing us to build interactive visualizations driven by widgets.